In [1]:
import numpy as np
import pandas as pd
import os

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("UYARI ENGELLEYİCİ")

UYARI ENGELLEYİCİ


In [2]:
filepath = '/kaggle/input/commonlit-evaluate-student-summaries'

prompts_test = pd.read_csv(os.path.join(filepath, 'prompts_test.csv'))
prompts_train = pd.read_csv(os.path.join(filepath, 'prompts_train.csv'))
sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
summaries_test = pd.read_csv(os.path.join(filepath, 'summaries_test.csv'))
summaries_train = pd.read_csv(os.path.join(filepath, 'summaries_train.csv'))
print("DOSYA YOL KODLARI")

DOSYA YOL KODLARI


In [3]:
print(summaries_train["text"].iloc[0])

The third wave was an experimentto see how people reacted to a new one leader government. It gained popularity as people wanted to try new things. The students follow anything that is said and start turning on eachother to gain higher power. They had to stop the experement as too many people got to radical with it blindly following there leader


In [4]:
desired_length = 20
filtered_summaries_train = pd.concat([group.sample(desired_length) for _, group in summaries_train.groupby('prompt_id')])

In [5]:
filtered_summaries_train

,student_id,prompt_id,text,content,wording
2171,4d67a6099fc8,39c16e,One element of a tragedy would b e that the pl...,-0.308448,0.048171
6666,eedaf3955aa0,39c16e,You must have a good plot not one that makes t...,0.376374,0.463619
1543,3795b81d1445,39c16e,A good tragedy should feature a change of for...,-1.204574,-1.169784
3211,71beea5e19f1,39c16e,"The 3 elements are the plot, the character, an...",0.205683,0.380538
4180,93c42389c68c,39c16e,Three elements of an ideal tragedy are focused...,-0.602425,-1.039843
...,...,...,...,...,...
6038,d746eb05063b,ebad26,The way's they dealt with spoiled meat is by f...,-0.731230,-0.833467
1653,3b6e444215dc,ebad26,"Paragraph 2 says that the factory would ""rub t...",-0.002466,-0.045439
3018,6afe80da9e2c,ebad26,The ways the factory would cover up the spoile...,0.404076,-0.755297
5920,d293370d610d,ebad26,the factories would can it or turn it into sau...,-0.393310,0.627128


# Verileri parçalara böl ki işin uzun sürmesin

# ****Tokenizasyon****

In [6]:
import nltk
# NLTK'den tokenizer'ı yükle
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenizasyon işlemi
filtered_summaries_train['tokenized_text'] = filtered_summaries_train['text'].apply(lambda x: word_tokenize(x))
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(lambda x: word_tokenize(x))
summaries_test['text'] = summaries_test['text'].apply(lambda x: word_tokenize(x))


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [7]:
print(filtered_summaries_train["tokenized_text"].iloc[3])

['The', '3', 'elements', 'are', 'the', 'plot', ',', 'the', 'character', ',', 'and', 'the', 'use', 'of', 'words', '.', 'It', 'should', 'be', 'something', 'people', 'can', 'relate', 'to', '.', 'The', 'story', 'should', 'be', 'interesting', '.', 'You', 'ca', "n't", 'do', 'this', 'without', 'the', 'character', ',', 'plot', 'and', 'words', '.', 'The', 'character', 'is', 'not', 'good', 'or', 'bad', 'but', 'ha', 'had', 'a', 'rough', 'life', '.', 'The', 'plot', 'is', 'relatable', 'to', 'the', 'reader', 'because', 'it', 'requres', 'a', 'falling', 'of', 'the', 'man', '.', 'The', 'words', 'bring', 'it', 'to', 'life', '.']


In [8]:
filtered_summaries_train["text"].iloc[3]

"The 3 elements are the plot, the character, and the use of words. It should be something people can relate to. The story should be interesting. You can't do this without the character, plot and words. The character is not good or bad but ha had a rough life. The plot is relatable to the reader because it requres a falling of the man. The words bring it to life."

In [9]:
filtered_summaries_train.head()

,student_id,prompt_id,text,content,wording,tokenized_text
2171,4d67a6099fc8,39c16e,One element of a tragedy would b e that the pl...,-0.308448,0.048171,"[One, element, of, a, tragedy, would, b, e, th..."
6666,eedaf3955aa0,39c16e,You must have a good plot not one that makes t...,0.376374,0.463619,"[You, must, have, a, good, plot, not, one, tha..."
1543,3795b81d1445,39c16e,A good tragedy should feature a change of for...,-1.204574,-1.169784,"[A, good, tragedy, should, feature, a, change,..."
3211,71beea5e19f1,39c16e,"The 3 elements are the plot, the character, an...",0.205683,0.380538,"[The, 3, elements, are, the, plot, ,, the, cha..."
4180,93c42389c68c,39c16e,Three elements of an ideal tragedy are focused...,-0.602425,-1.039843,"[Three, elements, of, an, ideal, tragedy, are,..."


# **Metin Temizliği**

In [10]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

# NLTK stopwords verilerini indirin
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Temizleme işlemlerini uygulama fonksiyonu
def preprocess_text(tokenized_text):
    # Her bir kelimeyi temizle
    cleaned_words = []
    for word in tokenized_text:
        # Küçük harfe dönüştürme
        word = word.lower()
        # Noktalama işaretlerini boşlukla değiştirme
        word = re.sub(r'[^\w\s]', ' ', word)
        # Gereksiz boşlukları kaldırma
        word = re.sub(r'\s+', ' ', word)
        # Stopwords'leri kaldırma
        if word not in stop_words:
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

# Tokenleştirilmiş metinleri temizleme
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['tokenized_text'].apply(preprocess_text)
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(preprocess_text)
summaries_test['text'] = summaries_test['text'].apply(preprocess_text)

# Noktalama işaretlerini boşlukla değiştirme
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('[^\w\s]', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('[^\w\s]', '')
summaries_test['text'] = summaries_test['text'].str.replace('[^\w\s]', '')

# ŞİMDİ BOŞLUKLARI KALDIRRRR HAYDİİİİİİİİİİİ
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('\s+', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('\s+', '')
summaries_test['text'] = summaries_test['text'].str.replace('\s+', '')

#Bişey deniyorum
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('  ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('  ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('  ', ' ')

filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('   ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('   ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('   ', ' ')

filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('    ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('    ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('    ', ' ')

# Temizlenmiş verileri gösterme
print(filtered_summaries_train['cleaned_text'])

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
2171    one element tragedy would b e plot starts good...
6666    must good plot one makes reader feel pity main...
1543    good tragedy feature change fortune prosperity...
3211    3 elements plot  character  use words  somethi...
4180    three elements ideal tragedy focused complex i...
                              ...                        
6038    way s dealt spoiled meat first would go take m...
1653    paragraph 2 says factory would  rub meat ons s...
3018    ways factory would cover spoiled meat  would p...
5920    factories would turn sauges  also put spoiled ...
7138    one way covered meat would rub soda hide smell...
Name: cleaned_text, Length: 80, dtype: object


In [11]:
print(filtered_summaries_train["cleaned_text"].iloc[3])

3 elements plot  character  use words  something people relate  story interesting  ca n t without character  plot words  character good bad ha rough life  plot relatable reader requres falling man  words bring life 


In [12]:
print(filtered_summaries_train["text"].iloc[3])

The 3 elements are the plot, the character, and the use of words. It should be something people can relate to. The story should be interesting. You can't do this without the character, plot and words. The character is not good or bad but ha had a rough life. The plot is relatable to the reader because it requres a falling of the man. The words bring it to life.


## ikinci aşama

In [13]:
# "text" sütunundaki tüm satırları birleştirip tek bir metin oluşturun
all_text = ' '.join(filtered_summaries_train['cleaned_text'])

# Metni boşluklardan bölerek kelimeleri ayırın ve bir liste oluşturun
words = all_text.split()

# Kelimelerin benzersiz (unique) olmasını sağlamak için bir küme (set) oluşturun
unique_words_list = list(set(words))

# Benzersiz kelimeleri görüntüleyin
print(len(unique_words_list))

1070


In [14]:
# 11. kelime ile 20. kelime arasındaki benzersiz kelimeleri görüntülemek için
print(unique_words_list[150:170])

['order', 'rubbing', 'donations', 'mixing', 'card', 'middel', 'realizing', 'isis', 'pickleing', 'ratting', 'quality', 'rich', 'men', 'experiment', 'lot', 'responsible', 'better', 'lay', 'say', 'depravity']


In [15]:
# Birden fazla kelimeyi düzeltmek için her bir düzeltmeyi ayrı satırlarda tanımlayın

corrections = {
    "structuree": "structure",
    "emploees": "employees",
    "governemtn": "government",
    "bellieved": "believed",
    "nuaseating":"nauseating",
    "extact":"extract",
    "valubule":"valuable",
    "reaso":"reason",
    "figting":"fighting",
    "complx":"complex",
    "sonstructed":"constructed",
    "differeny":"different",
    "instert":"insert",
    "experiemnt":"experiment",
    "folloeing":"following",
    "behaivior":"behavior",
    "beggng":"begging",
    "middl":"middle",
    "expirament":"experiment",
    "shoild":"should",
    "supposted":"supposed",
    "desissions":"decisions",
    "movemnet":"movement",
    "controllled":"controlled",
    "gorvernemt":"government",
    "afer":"after",
    "succesfull":"successful",
    "pharoaohs":"pharaohs",
    "pharoras":"pharaohs",
    "quaility":"quality",
    "tradedy":"tragedy",
    "wokers":"workers",
    "exampl":"example",
    "classś":"class",
    "includs":"includes",
    "susage":"sausage",
    "mobilty":"mobility",
    "saddness":"sadness",
    "sinckair":"sinclair",
    "artile":"article",
    "craftsmem":"craftsmen",
    "charactor":"character",
    "thorught":"through",
    "aviod":"avoid",
    "betweem":"between",
    "endof":"end of",
    "perceft":"perfect",
    "glycerince":"glycerine",
    "ful":"full"
}

# Her düzeltmeyi sırayla uygulayın
for old_word, new_word in corrections.items():
    filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace(old_word, new_word)

for old_word, new_word in corrections.items():
    filtered_summaries_train['text'] = filtered_summaries_train['text'].str.replace(old_word, new_word)

# Sütunun güncellendiğini kontrol etmek için ilk 10 satırı görüntüleyebilirsiniz
print("Başarılı")


Başarılı


In [16]:
filtered_summaries_train.head()

,student_id,prompt_id,text,content,wording,tokenized_text,cleaned_text
2171,4d67a6099fc8,39c16e,One element of a tragedy would b e that the pl...,-0.308448,0.048171,"[One, element, of, a, tragedy, would, b, e, th...",one element tragedy would b e plot starts good...
6666,eedaf3955aa0,39c16e,You must have a good plot not one that makes t...,0.376374,0.463619,"[You, must, have, a, good, plot, not, one, tha...",must good plot one makes reader feel pity main...
1543,3795b81d1445,39c16e,A good tragedy should feature a change of for...,-1.204574,-1.169784,"[A, good, tragedy, should, feature, a, change,...",good tragedy feature change fortune prosperity...
3211,71beea5e19f1,39c16e,"The 3 elements are the plot, the character, an...",0.205683,0.380538,"[The, 3, elements, are, the, plot, ,, the, cha...",3 elements plot character use words somethi...
4180,93c42389c68c,39c16e,Three elements of an ideal tragedy are focused...,-0.602425,-1.039843,"[Three, elements, of, an, ideal, tragedy, are,...",three elements ideal tragedy focused complex i...


# **BİR SONRAKİ TURDA LEMMATİZASYON**

In [17]:
#!pip install transformers
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification, DebertaConfig, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# ÇALIŞTIRMAYI BAŞARDIM AŞAĞIDAKİ KOD İLE.

Modeli yükle

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

# X ve y'yi ayırın
X = filtered_summaries_train['cleaned_text'].values
y = filtered_summaries_train[['content', 'wording']].values

# Model ve Tokenizer'ı yükleyin
model_name = '/kaggle/input/deberta-v3-small/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # İki etiket olduğunu varsayalım

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/deberta-v3-small/deberta-v3-small and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Verileri tokenize edin ve tensörlere çevirin
def tokenize_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer(
            text,
            padding="max_length",  # Metni maksimum token sayısına göre doldurun
            truncation=True,        # Uzun metinleri kırpın
            max_length=400,         # Maksimum token sayısını ayarlayın
            return_tensors="pt"    # PyTorch tensörleri olarak döndürün
        )
        
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0), torch.tensor(labels)

input_ids, attention_masks, labels = tokenize_data(X, y)

# Veriyi tensör veri kümesine dönüştürün
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(input_ids, attention_masks, labels)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [20]:
from sklearn.metrics import mean_squared_error

# Modeli eğitin
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train()
epochs = 5

for epoch in range(epochs):
    total_loss = 0
    step = 0  # Eğitim adımını izlemek için bir sayaç ekleyin
    
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Her adım sonunda ilerlemeyi yazdırın
        print(f"Epoch {epoch + 1}/{epochs} - Step {step + 1}/{len(dataloader)} - Loss: {loss.item():.4f}")
        step += 1
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}")

# İlk olarak content'i tahmin et
model.eval()
predictions_content = []
true_labels_content = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()
    true_labels_content.extend(labels.tolist())
    predictions_content.extend(predicted_labels)

# Şimdi wording'i tahmin et (aynı dataloader kullanılabilir)
model.eval()
predictions_wording = []
true_labels_wording = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()
    true_labels_wording.extend(labels.tolist())
    predictions_wording.extend(predicted_labels)

Epoch 1/5 - Step 1/3 - Loss: 0.2205
Epoch 1/5 - Step 2/3 - Loss: -0.1261
Epoch 1/5 - Step 3/3 - Loss: -0.3309
Epoch 1/5 - Loss: -0.0788
Epoch 2/5 - Step 1/3 - Loss: -0.1661
Epoch 2/5 - Step 2/3 - Loss: 0.0898
Epoch 2/5 - Step 3/3 - Loss: 0.0389
Epoch 2/5 - Loss: -0.0125
Epoch 3/5 - Step 1/3 - Loss: 0.0749
Epoch 3/5 - Step 2/3 - Loss: -0.2423
Epoch 3/5 - Step 3/3 - Loss: 0.2312
Epoch 3/5 - Loss: 0.0213
Epoch 4/5 - Step 1/3 - Loss: -0.2322
Epoch 4/5 - Step 2/3 - Loss: 0.2412
Epoch 4/5 - Step 3/3 - Loss: -0.1701
Epoch 4/5 - Loss: -0.0537
Epoch 5/5 - Step 1/3 - Loss: -0.2888
Epoch 5/5 - Step 2/3 - Loss: 0.1380
Epoch 5/5 - Step 3/3 - Loss: 0.1870
Epoch 5/5 - Loss: 0.0121


In [21]:
from sklearn.metrics import mean_squared_error

# İlk olarak content'i tahmin et
model.eval()
predictions_content = []
true_labels_content = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = logits.squeeze().tolist()  # Regresyon çıktısı olduğu için squeeze ve listeye çeviriyoruz
    true_labels_content.extend(labels.tolist())
    predictions_content.extend(predicted_labels)

# Şimdi wording'i tahmin et (aynı dataloader kullanılabilir)
model.eval()
predictions_wording = []
true_labels_wording = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = logits.squeeze().tolist()  # Regresyon çıktısı olduğu için squeeze ve listeye çeviriyoruz
    true_labels_wording.extend(labels.tolist())
    predictions_wording.extend(predicted_labels)

# MSE hesaplamak için
mse_content = mean_squared_error(true_labels_content, predictions_content)
mse_wording = mean_squared_error(true_labels_wording, predictions_wording)

print(f"Mean Squared Error (Content): {mse_content:.4f}")
print(f"Mean Squared Error (Wording): {mse_wording:.4f}")

Mean Squared Error (Content): 1.0049
Mean Squared Error (Wording): 1.0049


In [22]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Verileriniz
content_true = true_labels_content  # Content gerçek değerler
wording_true = true_labels_wording  # Wording gerçek değerler
content_pred = predictions_content   # Content tahmin değerler
wording_pred = predictions_wording   # Wording tahmin değerler

def compute_mcrmse(content_true, content_pred, wording_true, wording_pred):
    # Her iki sütun için RMSE hesaplamaları yapın
    content_rmse = np.sqrt(mean_squared_error(content_true, content_pred))
    wording_rmse = np.sqrt(mean_squared_error(wording_true, wording_pred))
    
    # Ortalama RMSE hesaplayın
    mcrmse = (content_rmse + wording_rmse) / 2.0
    
    return {
        "content_rmse": content_rmse,
        "wording_rmse": wording_rmse,
        "mcrmse": mcrmse,
    }

# MCRMSE sonuçlarını hesaplayın
mcrmse_result = compute_mcrmse(content_true, content_pred, wording_true, wording_pred)

# MCRMSE sonuçlarını yazdırın
print("Content RMSE:", mcrmse_result["content_rmse"])
print("Wording RMSE:", mcrmse_result["wording_rmse"])
print("MCRMSE:", mcrmse_result["mcrmse"])

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

# Compt_score hesaplama fonksiyonunu kullanarak sonuçları alın
compt_result = compt_score(content_true, content_pred, wording_true, wording_pred)

# Compt_score sonucunu yazdırın
print("Compt Score:", compt_result)

Content RMSE: 1.002429876302063
Wording RMSE: 1.002429876302063
MCRMSE: 1.002429876302063
Compt Score: 1.002429876302063


In [23]:
# Content ve wording tahminlerinin ortalamasını alın
average_content = np.mean(predictions_content)
average_wording = np.mean(predictions_wording)

# Mevcut "sub" dosyasına tahminleri ekleyin
sub['content'] = average_content
sub['wording'] = average_wording

# Güncellenmiş gönderim dosyasını kaydedin
sub.to_csv('submission.csv', index=False)